In [1]:
import time
import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp

# The IBM Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Session, Sampler


# Plotting functions
import matplotlib.pyplot as plt
import json
from qiskit_optimization.applications.knapsack import Knapsack
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp,to_ising
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator, Options
from qiskit_algorithms.optimizers import COBYLA,NFT
from qiskit_algorithms.minimum_eigensolvers import QAOA

In [2]:
service = QiskitRuntimeService()

Method to read the data from the kanpsack library

In [3]:
def read_data(filename):
    data_file = open(filename, 'r')
    count = 0
    p=[]
    w=[]
    while True:
        count += 1
        line=data_file.readline()
        if not line:
            break
        dp = [int(i) for i in line.rstrip('\n').split(' ')]
        if (count == 1):
            n=dp[0]
            b=dp[1]
        else:
            p.append(dp[0])
            w.append(dp[1])
    data_file.close()
    prob=Knapsack(p,w,b)
    qp=prob.to_quadratic_program()
    return qp 

In [4]:
data="kp_5_80"
qp=read_data('../../'+data)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Knapsack

Maximize
 obj: 33 x_0 + 24 x_1 + 36 x_2 + 37 x_3 + 12 x_4
Subject To
 c0: 15 x_0 + 20 x_1 + 17 x_2 + 8 x_3 + 31 x_4 <= 80

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4
End



In [5]:
qb=QuadraticProgramToQubo().convert(qp)
op,of = qb.to_ising()
min_qubits=op.num_qubits

In [6]:
from qiskit_optimization.algorithms.cplex_optimizer import CplexOptimizer
solution=CplexOptimizer().solve(qp)

In [7]:
solution.fval

130.0

In [8]:
print(solution.prettyprint())

objective function value: 130.0
variable values: x_0=1.0, x_1=1.0, x_2=1.0, x_3=1.0, x_4=0.0
status: SUCCESS


In [9]:
lbdevice=service.least_busy(simulator=False,min_num_qubits=min_qubits)

In [10]:
lbdevice.name

'ibm_nazca'

In [ ]:
device=lbdevice.name
backend = service.get_backend(device)
optimizer = COBYLA(maxiter=100)
options=Options()
options.optimization_level=3
options.resilience_level=1
options.execution.shots=2048
stime=time.time()
with Session(service=service, backend=backend):
    sampler = Sampler(options=options)
    qaoa = QAOA(sampler, optimizer, reps=1)
    result = qaoa.compute_minimum_eigenvalue(op)
total_time=time.time()-stime
best_bts = result.best_measurement['bitstring'][::-1]

In [74]:
print(optimizer.setting)

Optimizer: COBYLA
-- method: cobyla
-- bounds_support_level: 1
-- gradient_support_level: 1
-- initial_point_support_level: 3
-- options: {'maxiter': 100, 'disp': False, 'rhobeg': 1.0}
-- max_evals_grouped: 1
-- kwargs: {'tol': None}



In [61]:
#knapsack_results={}
td={}
td['data_set']=data
td['opt']=solution.fval
td['device']=device
td['optimization_level']=options.optimization_level
td['resilience_level']=options.resilience_level
td['shots']=options.execution.shots
td['total time']=total_time
td['bm_value']=np.real(-of-result.best_measurement['value'])
td['bm_bitstring']=result.best_measurement['bitstring']
td['bm_probability']=result.best_measurement['probability']
td['cost_function_evals']=result.cost_function_evals
td['optimizer_time']=result.optimizer_time